# Get data for MA

In [4]:
#import packages
from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from shapely.geometry import Point
# from adjustText import adjust_text

In [15]:
#sample code from https://github.com/teamookla/ookla-open-data/blob/master/tutorials/aggregate_by_county_py.ipynb
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_fixed_tiles.zip"
    return url

In [16]:
tile_url = get_tile_url("fixed", 2020, 1)
tile_url

'https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance/type%3Dfixed/year%3D2020/quarter%3D1/2020-01-01_performance_fixed_tiles.zip'

In [17]:
tiles = gp.read_file(tile_url)
tiles

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry
0,3100121113233332,16599,13312,70,37,27,"POLYGON ((106.68823 -6.30984, 106.69373 -6.309..."
1,3112231300122103,42461,13516,19,171,53,"POLYGON ((145.04700 -37.82714, 145.05249 -37.8..."
2,0331102110022021,25765,15185,16,228,100,"POLYGON ((-9.13513 38.71123, -9.12964 38.71123..."
3,1202020213110310,79703,15579,18,199,32,"POLYGON ((1.35132 51.38550, 1.35681 51.38550, ..."
4,1231210133132220,27032,24622,23,174,55,"POLYGON ((75.84961 30.91165, 75.85510 30.91165..."
5,0231301222133033,166797,55407,18,38,10,"POLYGON ((-98.11340 29.71191, -98.10791 29.711..."
6,1322301113030321,31712,33910,5,215,62,"POLYGON ((106.63879 10.71459, 106.64429 10.714..."
7,0302222300132222,152459,113630,7,53,18,"POLYGON ((-88.33008 41.90637, -88.32458 41.906..."
8,1233003111321212,26441,25395,7,519,123,"POLYGON ((73.00415 19.03616, 73.00964 19.03616..."
9,0313131011130330,110506,15818,17,106,37,"POLYGON ((-1.46118 52.40912, -1.45569 52.40912..."


In [8]:
tiles.head()

NameError: name 'tiles' is not defined

In [5]:

# zipfile of U.S. county boundaries
county_url = "https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip" 
counties = gp.read_file(county_url)

In [6]:
ma_counties = counties.loc[counties['STATEFP'] == '25'].to_crs(epsg=4326)

In [7]:
ma_counties.crs

{'init': 'epsg:4326', 'no_defs': True}

In [26]:
tiles_in_ma_counties = gp.sjoin(tiles, ma_counties, how="inner", op='intersects')

In [1]:
tiles_in_ma_counties.head()

NameError: name 'tiles_in_ma_counties' is not defined

In [30]:
tiles_in_ma_counties.columns

Index(['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
       'geometry', 'index_right', 'STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID',
       'NAME', 'NAMELSAD', 'LSAD', 'CLASSFP', 'MTFCC', 'CSAFP', 'CBSAFP',
       'METDIVFP', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON'],
      dtype='object')

In [34]:
ma_data_q1_2020 = tiles_in_ma_counties.filter(items = ['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
       'geometry', 'STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAMELSAD'])

In [37]:
i = 1
tiles_in_ma_counties.to_csv(path_or_buf= f'./ookla2020q{i}ma.csv', 
                                columns = ['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
       'geometry', 'STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAMELSAD'])
# ma_data_q1_2020.to_csv(path_or_buf= './ookla2020q1ma.csv')

In [36]:
#do all the same for q2 q3 q4
def get_data_to_csv(service_type: str, year: int, q: int):
    tile_url = get_tile_url(service_type, year, q)
    tiles = gp.read_file(tile_url)
    tiles_in_ma_counties = gp.sjoin(tiles, ma_counties, how="inner", op='intersects')
    tiles_in_ma_counties.to_csv(path_or_buf= f'./ookla2020q{q}ma.csv', 
                                columns = ['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
       'geometry', 'STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAMELSAD'])

In [ ]:
for q in range(2,5):
    get_data_to_csv("fixed", 2020, q)
    

In [10]:
print('k',1)

k 1
